In [1]:
import os
os.chdir('../scripts/')
from sentinel2 import FarmKML
from pprint import pprint as pp
import rasterio

In [2]:
votm = FarmKML()


    Name                  Description
--  --------------------  ---------------------------------
 0  Non-grazing
 1  Farm boundary
 2  Paddock 1
 3  Paddock 2
 4  Paddock 3
 5  Paddock 4
 6  Paddock 5
 7  Paddock 6
 8  Paddock 7
 9  Paddock 8
10  Paddock 9
11  Paddock 10
12  Paddock 11
13  Paddock 12
14  Paddock 13
15  Paddock 14
16  Paddock 15
17  Paddock 16
18  Paddock 17
19  Paddock 18
20  Paddock 19
21  Paddock 20
22  Paddock 21
23  Paddock 22
24  Paddock 23
25  East 18 North third
26  East 18 South third
27  East 18 Middle third
28  buffered              Max extents of dataframe with pad
Enter your choice for outline of grazeable land:  1

    Name                  Description
--  --------------------  ---------------------------------
 0  Non-grazing
 1  Farm boundary
 2  Paddock 1
 3  Paddock 2
 4  Paddock 3
 5  Paddock 4
 6  Paddock 5
 7  Paddock 6
 8  Paddock 7
 9  Paddock 8
10  Paddock 9
11  Paddock 10
12  Paddock 11
13  Paddock 12
14  Paddock 13
15  Paddock 14
16  Paddock 

In [3]:
votm_m = votm.grazeable_gdf.to_crs(crs='epsg:32616')

In [4]:
votm.grazeable_gdf[votm.grazeable_gdf.Name == 'buffered'].geometry.bounds

,minx,miny,maxx,maxy
26,-89.158188,42.159,-89.133337,42.18186


# Code for trimming the jp2 file and saving a smaller geotiff

In [5]:
shape = votm_m[votm_m.Name == 'buffered'].geometry

with rasterio.open("../T16TCM_20200917T163839_B04_10m.jp2") as src:
    out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
    out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open("cropped_masked.tif", "w", **out_meta) as dest:
    dest.write(out_image)